In [ ]:
import collections
import random


EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

curve = EllipticCurve(
    'secp256k1',
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141,
    # Subgroup cofactor.
    h=1,
)

# Modular arithmetic ##########################################################


def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p


# Functions that work on curve points #########################################


def is_on_curve1(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0


def point_add(point1, point2):
    """Returns the result of point1 + point2 according to the group law."""
    assert is_on_curve1(point1)
    assert is_on_curve1(point2)

    if point1 is None:
        # 0 + point2 = point2
        return point2
    if point2 is None:
        # point1 + 0 = point1
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        # point1 + (-point1) = 0
        return None

    if x1 == x2:
        # This is the case point1 == point2.
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p, -y3 % curve.p)

    assert is_on_curve1(result)

    return result


def scalar_mult(k, point):
    """Returns k * point computed using the double and point_add algorithm."""
    assert is_on_curve1(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        # k * point = -k * (-point)
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)

        k >>= 1

    assert is_on_curve1(result)

    return result


# Keypair generation

def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)
    return private_key, public_key
print("Basepoint:\t", curve.g)

aliceSecretKey, alicePublicKey = make_keypair()
bobSecretKey, bobPublicKey = make_keypair()

print("Alice\'s secret key:\t", aliceSecretKey)
print("Alice\'s public key:\t", alicePublicKey)
print("Bob\'s secret key:\t", bobSecretKey)
print("Bob\'s public key:\t", bobPublicKey)
sharedSecret1 = scalar_mult(bobSecretKey, alicePublicKey)
sharedSecret2 = scalar_mult(aliceSecretKey, bobPublicKey)
print("Alice\'s shared key:\t", sharedSecret1)
print("Bob\'s shared key:\t", sharedSecret2)
print("The shared value is the x-value: \t", (sharedSecret1[0]))

Basepoint:	 (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
Alice's secret key:	 103163957511965503687409664299982520879488610620759255833559644727832895156238
Alice's public key:	 (5257818465961244799758083809767335623379701928485252643447610405837719172250, 86264882505114570302929747416728210100420107485849011942341808218573351387139)
Bob's secret key:	 43850246553352343151917562499257200915853558051177502689315691959741704572887
Bob's public key:	 (46167490462098588135040808535352842622484892248816923211081209916414104350167, 107964918931759882976318183194514771602128202346317292403000033649399275225525)
Alice's shared key:	 (82042917435183972918038248049741832565175958477289260772975848088818576369895, 42379319143013012389613827177121537789856780201453135900441102186604579659919)
Bob's shared key:	 (82042917435183972918038248049741832565175958477289260772975848088818576369

In [ ]:
def add_points(P,Q,p):
    x1,y1=P
    x2,y2=Q
    if x1==x2 and y1==y2 :
        beta=(3*x1*x2+a)*pow(2*y1,-1,p)
    else:
        beta=(y2-y1)*pow(x2-x1,-1,p)

    x3=(beta*beta-x1-x2)%p
    y3=(beta*(x1-x3)-y1)%p

    is_on_curve((x3,y3),p)
    return x3,y3

def is_on_curve(P,p):
    x,y=P
    assert (y*y)%p==( pow(x, 3, p) + a*x + b )% p

def apply_double_and_add_method(G, k, p) :
    target_point = G
    k_binary=bin(k)[2:]
    for i in range(1, len(k_binary) ) :
        current_bit = k_binary [i: i+1]
        # doubling — always
        target_point = add_points(target_point, target_point, p)

        if current_bit=="1":
            target_point = add_points (target_point, G, p)
    is_on_curve(target_point,p)

    return  target_point

In [ ]:
a=0;b=7
G=(55066263022277343669578718895168534326250603453777594175500187360389116729240,32670510020758816978083085130507043184471273380659243275938904335757337482424)
#finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [ ]:
is_on_curve(G,p)

TypeError: is_on_curve() takes 1 positional argument but 2 were given

In [ ]:
#point_2G=add_points(G,G,p)
tmp_point = G
for i in range(2, 21):
    tmp_point= add_points ( tmp_point,G,p)
    print (f"{i}G ={ tmp_point}")



TypeError: is_on_curve() takes 1 positional argument but 2 were given

In [ ]:
apply_double_and_add_method(G=G,k=20,p=p)

TypeError: is_on_curve() takes 1 positional argument but 2 were given

In [ ]:
is_on_curve(G,p)

TypeError: is_on_curve() takes 1 positional argument but 2 were given

In [ ]:
# alice
ka = aliceSecretKey
Qa = apply_double_and_add_method(G = G, k = ka,p= p)
# bob
m= 43424217

s = apply_double_and_add_method(G = G, k =m,p= p)

r = random.getrandbits(128)
c1 = apply_double_and_add_method(G = G, k =r,p= p)

c2= apply_double_and_add_method(G = Qa, k = r, p=p)
c2 = add_points(c2, s, p)
(c1,c2)
# s_prime = c2 — ka x c1
# s_prime = c2 + ( ka x —c1)
c1_prime = (c1[0], (-1*c1[1]) %p)

s_prime=apply_double_and_add_method(G=c1_prime,k=ka,p=p)
s_prime=add_points(P=c2,Q=s_prime,p=p)
s_prime
s

TypeError: is_on_curve() takes 1 positional argument but 2 were given

In [ ]:
# s_prime = c2 — ka x c1
# s_prime = c2 + ( ka x —c1)# bob
m= 43424217

s = apply_double_and_add_method(G = G, k =m,p= p)

r = random.getrandbits(128)
c1 = apply_double_and_add_method(G = G, k =r,p= p)

c2= apply_double_and_add_method(G = Qa, k = r, p=p)
c2 = add_points(c2, s, p)

In [ ]:
(c1,c2)

((23725333115428360688015298090970188190606353582556142028646884238399621259408,
  89567990616352906010840154564832653935216699011689374436766828202779474316392),
 (47477410512958114111889091417057196908327728674125014915307428435671331366014,
  92811820448017974089686691029581902148739026870251962074423989532970549114596))

In [ ]:
c1_prime = (c1[0], (-1*c1[1]) %p)

In [ ]:
s_prime=apply_double_and_add_method(G=c1_prime,k=ka,p=p)
s_prime=add_points(P=c2,Q=s_prime,p=p)

In [ ]:
s_prime

(61923371908307645681599963956552275781958346742043893176124343276692223439156,
 96861786012748632420154169891607477506648868670246826739860922798413865027620)

In [ ]:
s

(61923371908307645681599963956552275781958346742043893176124343276692223439156,
 96861786012748632420154169891607477506648868670246826739860922798413865027620)